<a href="https://colab.research.google.com/github/timothymwenda/Hero-privacy-policy/blob/main/certverification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Define coordinates and fonts for text placement
# You can adjust these coordinates (x, y) to fine-tune the position of the text on the template
name_x, name_y = 378, 358  # Adjusted name position
name_font = 'Helvetica'
name_font_size = 20

course_x, course_y = 378, 298 # Adjusted course position
course_font = 'Helvetica'
course_font_size = 20

date_x, date_y = 378, 228 # Adjusted date position
date_font = 'Helvetica'
date_font_size = 20

# Assuming 'Issue Date' should also be at a specific position,
# but it wasn't provided in the latest coordinates, I'll keep the previous one
# or you can provide the coordinate for 'Issue Date' as well.
issue_date_x, issue_date_y = 150, 620
issue_date_font = 'Helvetica'
issue_date_font_size = 20

# Define coordinates and font for "reg number" - set these manually
reg_number_x, reg_number_y = 540, 126  # Example manual position, adjust as needed
reg_number_font = 'Helvetica'
reg_number_font_size = 10

# Define coordinates and font for "serial number" - set these manually
serial_number_x, serial_number_y = 100, 500  # Example manual position, adjust as needed
serial_number_font = 'Helvetica'
serial_number_font_size = 10

# Define coordinates for "qr code" - set these manually
qr_code_x, qr_code_y = 300, 80  # Example manual position, adjust as needed

In [10]:
%pip install pandas PyPDF2 qrcode openpyxl

In [26]:
from PyPDF2 import PdfWriter, PdfReader
from io import BytesIO
import os
from reportlab.pdfgen import canvas
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
import qrcode # Import the qrcode library

# Create 'certificates' directory if it doesn't exist
if not os.path.exists('certificates'):
    os.makedirs('certificates')

# Open the template PDF outside the loop for efficiency
template_path = 'template.pdf'


# Get template page dimensions from the template and convert to float for calculations
# We can get these once outside the loop as the template size is constant
template_reader_initial = PdfReader(template_path)
template_page_initial = template_reader_initial.pages[0]
page_width = float(template_page_initial.mediabox.width)
page_height = float(template_page_initial.mediabox.height)


# Convert DataFrame to a list of dictionaries for potentially more stable iteration
student_data = df.to_dict('records')

# Iterate through each student record with an index to generate serial numbers
for index, row_dict in enumerate(student_data):
    # Create a new BytesIO object to store the modified page
    packet = BytesIO()

    # Create a new PDF with student information using ReportLab
    can = canvas.Canvas(packet, pagesize=(page_width, page_height)) # Use template size for canvas

    # Format the date
    try:
        # Assuming the date format in the Excel is like '9th January 2025'
        # You might need to adjust the format string if your dates are different
        date_obj = datetime.strptime(row_dict['Date'], '%dth %B %Y')
        formatted_date = date_obj.strftime('%B %d, %Y')
    except (ValueError, TypeError):
        # Fallback for other date formats or if the column is missing/has errors
        try:
            date_obj = datetime.strptime(str(row_dict['Date']), '%Y-%m-%d %H:%M:%S') # Handle datetime objects from pandas
            formatted_date = date_obj.strftime('%B %d, %Y')
        except (ValueError, TypeError):
             formatted_date = str(row_dict['Date'])# Use original date as string if formatting fails


    # Calculate centered x-coordinate for Name, Course, and Date
    name_text = f"{row_dict['Name ']}" # Using 'Name ' with space
    name_text_width = stringWidth(name_text, name_font, name_font_size)
    name_x_centered = (page_width - name_text_width) / 2

    course_text = f"{row_dict['Course ']}" # Using 'Course ' with space
    course_text_width = stringWidth(course_text, course_font, course_font_size)
    course_x_centered = (page_width - course_text_width) / 2

    date_text = f"{formatted_date}"
    date_text_width = stringWidth(date_text, date_font, date_font_size)
    date_x_centered = (page_width - date_text_width) / 2

    # Set font and draw text for each field without tags, using centered x-coordinates for Name, Course, Date
    can.setFont(name_font, name_font_size)
    can.drawString(name_x_centered, name_y, name_text)

    can.setFont(course_font, course_font_size)
    can.drawString(course_x_centered, course_y, course_text)

    can.setFont(date_font, date_font_size)
    can.drawString(date_x_centered, date_y, date_text)

    # Draw "Reg No" using manual coordinates
    can.setFont(reg_number_font, reg_number_font_size)
    can.drawString(reg_number_x, reg_number_y, f"{row_dict['Reg No.']}")

    # --- Add Serial Number and QR Code ---

    # Use data from the "Sr No" column as the serial number
    serial_number = str(row_dict['Sr No'])

    # Draw the serial number using manual coordinates
    can.setFont(serial_number_font, serial_number_font_size)
    can.drawString(serial_number_x, serial_number_y, f"Serial No: {serial_number}")

    # Generate QR code
    # You can customize the data encoded in the QR code.
    # For example, you could include a link to a verification page with the serial number.
    qr_data = f"Certificate Serial Number: {serial_number} | Name: {row_dict['Name ']} | Course: {row_dict['Course ']}"
    qr = qrcode.make(qr_data)

    # Save the QR code to a temporary file
    qr_code_filename = f"temp_qr_{index}.png"
    qr.save(qr_code_filename)

    # Draw the QR code on the canvas using manual coordinates
    # You might need to adjust the size of the QR code image
    qr_code_size = 100 # Example size in points, adjust as needed
    can.drawImage(qr_code_filename, qr_code_x, qr_code_y, width=qr_code_size, height=qr_code_size)

    # Remove the temporary QR code file
    os.remove(qr_code_filename)

    can.save()

    # Move the cursor to the beginning of the BytesIO object
    packet.seek(0)
    new_content_pdf = PdfReader(packet)

    # Create a PdfWriter object for the output
    output_pdf_writer = PdfWriter()

    # Create a new PdfReader for the template inside the loop for each certificate
    template_reader_loop = PdfReader(template_path)
    template_page_loop = template_reader_loop.pages[0]

    # Merge the new PDF content onto the template page
    template_page_loop.merge_page(new_content_pdf.pages[0])

    # Add the merged page to the output PdfWriter
    output_pdf_writer.add_page(template_page_loop)

    # Create a filename for the output PDF
    output_filename = f"certificates/{row_dict['Name '].replace(' ', '_')}_certificate.pdf" # Using 'Name ' with space

    # Write the content to the output file
    with open(output_filename, 'wb') as output_file:
        output_pdf_writer.write(output_file)

In [15]:
import pandas as pd

df = pd.read_excel('students.xlsx')
display(df.head())

,Sr No,Reg No.,Name,Course,Date
0,CTL2411001,CTL2411002,Vincent Kitavi,Power BI Data Analyst Associate,9th January 2025
1,LTL2501001,LTL2501001,Ishan Dave,Power BI Data Analyst Associate,9th January 2025
2,LTL2501002,LTL2501002,Tabitha Ngei,Microsoft azure Administrator Associate,17th January 2025
3,LTL2501004,LTL2501004,Brenda Njeri,CompTIA Security+,24th January 2025
4,LTL2501005,LTL2501005,Abdulaziz ibrahim,Certified Cybersecurity Technician,31st January 2025
